In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, preprocessing
from sklearn.preprocessing import Imputer, PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold, KFold
from keras.layers import Dense, Activation, LSTM
from keras.models import Sequential
from keras.regularizers import l2, activity_l2
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from joblib import Parallel, delayed

Using Theano backend.


In [2]:
#Reading the data, into a Data Frame.
Data = pd.read_csv('/home/prajwal/Desktop/bank-additional/bank-additional-full.csv',delimiter=';',header=0)

#Encoding the data, encoding the string values into numerical values.
encode = preprocessing.LabelEncoder()

#Selcting the columns of string data type
names = Data.select_dtypes(include=['object'])

#Function that encodes the string values to numerical values.
def enc(data,column):
    data[column] = encode.fit_transform(data[column])
    return data
for column in names:
        Data = enc(Data,column)

#The list of second level models.
second_level_model_list=['Stack','Blend']

#Initializing the test dataset.
test = pd.DataFrame()

#Initializing two data frames that will be used as training data for the stacked model.
stack_X = pd.DataFrame() #The data frame will contain the predictions of the base models.
stack_Y = pd.DataFrame() #The data frame will contain the calss labels of the base models.

#Initializing two data frames that will be used as training data for the blending model.
blend_X = pd.DataFrame() #The data frames will contain the predictions and raw features  of the base models.
raw_features_X = pd.DataFrame() #The data frames will contain the raw features  of the data, which will be concatenated with the predictions.

#Initializing the dataframes that will be used for testing the stacking and blending models.
test_blend_X = pd.DataFrame()
test_raw_features_X = pd.DataFrame()
test_stack_X = pd.DataFrame()
test_stack_Y = pd.DataFrame()

In [3]:
#Splitting the data into training and testing datasets (Stratified Split)
def data_split():
    global Data
    global test
    Data, test = train_test_split(Data, test_size = 0.1,stratify=Data['y'])

In [4]:
#This function is used to convert the predictions of the base models into a DataFrame.
def build_data_frame(data):
    data_frame = pd.DataFrame(data).T
    return data_frame

In [5]:
#Defining the parameters for the XGBoost (Gradient Boosting) Algorithm.
def param_set():
    #Gradient Boosting (XGBoost)
    param = {}
    #Setting Parameters for the Booster
    param['booster'] = 'gbtree'
    param['objective'] = 'binary:logistic'
    param["eval_metric"] = "auc"
    param['eta'] = 0.3
    param['gamma'] = 0
    param['max_depth'] = 6
    param['min_child_weight'] = 1
    param['max_delta_step'] = 0
    param['subsample'] = 1
    param['colsample_bytree'] = 1
    param['silent'] = 1
    param['seed'] = 0
    param['base_score'] = 0.5
    param['lambda_bias'] = 1
    return param

In [6]:
#Initializing the base models.
gradient_boosting=xgb.Booster()
multi_layer_perceptron = Sequential()
decision_tree = DecisionTreeClassifier(max_depth = 6)
random_forest = RandomForestClassifier()
linear_regression = linear_model.LinearRegression()
logistic_regression_L1 = linear_model.LogisticRegression(penalty = 'l1')
logistic_regression_L2 = linear_model.LogisticRegression(penalty = 'l2')

#Initializing the second level models.
stack=xgb.Booster()
blend=xgb.Booster()

In [7]:
#Trains the Gradient Boosting model.
def train_gradient_boosting(train_X,train_Y):

    param = param_set()
    dtrain = xgb.DMatrix(train_X,label=train_Y)
    gradient_boosting = xgb.train(param, dtrain)
    return gradient_boosting

In [8]:
#Trains the Multi Layer Perceptron model.
def train_multi_layer_perceptron(train_X,train_Y):
    
    multi_layer_perceptron.add(Dense(output_dim = 64, input_dim = 20, init = 'uniform', activation = 'sigmoid'))
    multi_layer_perceptron.add(Dense(output_dim = 1, input_dim = 64,activation = 'sigmoid',))
    multi_layer_perceptron.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy',metrics = ['accuracy'])
    multi_layer_perceptron.fit(train_X.as_matrix(), train_Y.as_matrix(), nb_epoch = 5, batch_size = 128)
    return multi_layer_perceptron

In [9]:
#Trains the Decision Tree model.
def train_decision_tree(train_X,train_Y):
    
    decision_tree.fit(train_X,train_Y)
    return decision_tree

In [10]:
#Trains the Random Forest model.
def train_random_forest(train_X,train_Y):
    
    random_forest.fit(train_X,train_Y)
    return random_forest

In [11]:
#Trains the Linear Regression model.
def train_linear_regression(train_X,train_Y):
    
    #Scaling the data
    train_X = preprocessing.StandardScaler().fit_transform(train_X)
    linear_regression.fit(train_X,train_Y)
    return linear_regression

In [12]:
#Trains the Logistic Regression (L1) model.
def train_logistic_regression_L1(train_X,train_Y):
    
    #Scaling the data
    train_X = preprocessing.StandardScaler().fit_transform(train_X)
    logistic_regression_L1.fit(train_X,train_Y)
    return logistic_regression_L1

In [13]:
#Trains the Logistic Regression (L1) model.
def train_logistic_regression_L2(train_X,train_Y):
    
    #Scaling the data
    train_X = preprocessing.StandardScaler().fit_transform(train_X)
    logistic_regression_L2.fit(train_X,train_Y)
    return logistic_regression_L2   

In [14]:
#Trains the Stacking model (Gradient Boosting - XGBoost)
def train_stack_model(train_X,train_Y):
    
    param = param_set()
    dtrain = xgb.DMatrix(train_X,label = train_Y)
    stack = xgb.train(param, dtrain)
    return stack

In [15]:
#Trains the blending model (Gradient Boosting - XGBoost)
def train_blend_model(train_X,train_Y): 
    
    param = param_set()
    dtrain = xgb.DMatrix(train_X,label = train_Y)
    blend = xgb.train(param, dtrain)
    return blend

In [16]:
def cross_val_gradient_boosting(cross_val_X,cross_val_Y):
    
    predict = gradient_boosting.predict(xgb.DMatrix(cross_val_X,label=cross_val_Y))
    auc = roc_auc_score(cross_val_Y,predict)
    return [auc,predict]

In [17]:
def cross_val_multi_layer_perceptron(cross_val_X,cross_val_Y):
    
    predict = multi_layer_perceptron.predict_on_batch(cross_val_X)
    auc = roc_auc_score(cross_val_Y,predict)
    return [auc,predict]

In [18]:
def cross_val_decision_tree(cross_val_X,cross_val_Y):
    
    global decision_tree
    predict = decision_tree.predict_proba(cross_val_X)[:,1]
    auc = roc_auc_score(cross_val_Y,predict)
    return [auc,predict]

In [19]:
def cross_val_random_forest(cross_val_X,cross_val_Y):
    
    predict = random_forest.predict_proba(cross_val_X)[:,1]
    auc = roc_auc_score(cross_val_Y,predict)
    return [auc,predict]

In [20]:
def cross_val_linear_regression(cross_val_X,cross_val_Y):
    
    cross_val_X=preprocessing.StandardScaler().fit_transform(cross_val_X)
    predict = linear_regression.predict(cross_val_X)
    auc = roc_auc_score(cross_val_Y,predict)
    return [auc,predict]

In [21]:
def cross_val_logistic_regression_L1(cross_val_X,cross_val_Y):
    
    cross_val_X=preprocessing.StandardScaler().fit_transform(cross_val_X)
    predict = logistic_regression_L1.predict_proba(cross_val_X)[:,1]
    auc = roc_auc_score(cross_val_Y,predict)
    return [auc,predict]

In [22]:
def cross_val_logistic_regression_L2(cross_val_X,cross_val_Y):
    
    cross_val_X=preprocessing.StandardScaler().fit_transform(cross_val_X)
    predict = logistic_regression_L2.predict_proba(cross_val_X)[:,1]
    auc = roc_auc_score(cross_val_Y,predict)
    return [auc,predict]

In [23]:
def cross_val_stack(cross_val_X,cross_val_Y):

    predict = stack.predict(xgb.DMatrix(cross_val_X,label=cross_val_Y))
    auc = roc_auc_score(cross_val_Y,predict)
    return [auc,predict]

In [24]:
def cross_val_blend(cross_val_X,cross_val_Y):

    predict = blend.predict(xgb.DMatrix(cross_val_X,label=cross_val_Y))
    auc = roc_auc_score(cross_val_Y,predict)
    return [auc,predict]

In [25]:
#Initialzing the variables that will be used to calculate the area under the curve. (cross Validation Data)
metric_linear_regression=list()
#avg_linear_regeression=0
metric_logistic_regression_L2=list()
#avg_logistic_regression_L2=0
metric_logistic_regression_L1=list()
#avg_logistic_regression_L1=0
metric_decision_tree=list()
#avg_decision_tree=0
metric_random_forest=list()
#avg_random_forest=0
metric_multi_layer_perceptron=list()
#avg_multi_layer_perceptron=0
metric_gradient_boosting=list()
#avg_gradient_boosting=0

In [26]:
#Initialzing the variables that will be used to calculate the area under the curve. (Test Data)
test_metric_linear_regression=0
test_metric_logistic_regression_L2=0
test_metric_logistic_regression_L1=0
test_metric_decision_tree=0
test_metric_random_forest=0
test_metric_multi_layer_perceptron=0
test_metric_gradient_boosting=0
test_metric_stacking=0
test_metric_blending=0

In [27]:
#The list of base model functions (Training).
train_base_model_list=[train_gradient_boosting,train_multi_layer_perceptron,train_decision_tree,train_random_forest,
                 train_linear_regression,train_logistic_regression_L1,train_logistic_regression_L2]

#The list of base model functions (Cross Validation).
cross_val_base_model_list=[cross_val_gradient_boosting,cross_val_multi_layer_perceptron,cross_val_decision_tree,cross_val_random_forest,
                 cross_val_linear_regression,cross_val_logistic_regression_L1,cross_val_logistic_regression_L2]

#The list of second level model functions.
cross_val_second_level_model=[cross_val_stack,cross_val_blend]

In [28]:
def train_cross_val_base_models():
    
    #Cross Validation using Stratified K Fold
    train, cross_val = train_test_split(Data, test_size = 0.5,stratify=Data['y'])
    
    #Training the base models, and calculating AUC on the cross validation data.
    #Selecting the data (Traing Data & Cross Validation Data)
    train_Y=train['y']
    train_X=train.drop(['y'],axis=1)
    cross_val_Y=cross_val['y']
    cross_val_X=cross_val.drop(['y'],axis=1)
    
    global gradient_boosting
    global multi_layer_perceptron
    global decision_tree
    global random_forest
    global linear_regression
    global logistic_regression_L1
    global logistic_regression_L2

    #Training the base models parallely, the resulting models are stored which will be used for cross validation.
    [gradient_boosting,multi_layer_perceptron,decision_tree,random_forest,linear_regression,logistic_regression_L1,logistic_regression_L2]=(Parallel(n_jobs=-1)(delayed(function)(train_X, train_Y) for function in train_base_model_list))
    
    #Computing the AUC and Predictions of all the base models on the cross validation data parallely.
    auc_predict_cross_val=(Parallel(n_jobs=-1)(delayed(function)(cross_val_X,cross_val_Y) for function in cross_val_base_model_list))
    
    #Gradient Boosting (XGBoost)
    #The AUC error (Cross Validation Data)
    auc,predict_gradient_boosting=auc_predict_cross_val[0][0],auc_predict_cross_val[0][1]
    metric_gradient_boosting.append(auc)
    
    #Multi Layer Perceptron
    #The AUC (Cross Validation Data)
    predict_mlp=list()
    auc,predict_multi_layer_perceptron=auc_predict_cross_val[1][0],auc_predict_cross_val[1][1]
    metric_multi_layer_perceptron.append(auc)
    
    #predict_multi_layer_perceptron returns a list of lists containing the predictions, this cannot be converted to a dataframe.
    #This inner lists are converted to floats and then used to convert it to a dataframe.
    for i in predict_multi_layer_perceptron:
        predict_mlp.append(float(i))
    
    #Decision Tree)
    #The AUC (Cross Validation Data)
    auc,predict_decision_tree=auc_predict_cross_val[2][0],auc_predict_cross_val[2][1]
    metric_decision_tree.append(auc)
    
    #Random Forest (Deafult=10 Trees)
    #The AUC (Cross Validation Data)
    auc,predict_random_forest=auc_predict_cross_val[3][0],auc_predict_cross_val[3][1]
    metric_random_forest.append(auc)
    
    #Linear Regression
    #The AUC (Cross Validation Data)
    auc,predict_linear_regression=auc_predict_cross_val[4][0],auc_predict_cross_val[4][1]
    metric_linear_regression.append(auc)
    
    #Logistic Regression (Default=l2)
    #The AUC (Cross Validation Data)
    auc,predict_logistic_regression_L1=auc_predict_cross_val[5][0],auc_predict_cross_val[5][1]
    metric_logistic_regression_L1.append(auc)
    
    #Logistic Regression-L2
    #The AUC (Cross Validation Data)
    auc,predict_logistic_regression_L2=auc_predict_cross_val[6][0],auc_predict_cross_val[6][1]
    metric_logistic_regression_L2.append(auc)
    
    #Building a list that contains all the predictions of the base models.
    predict_list=[predict_gradient_boosting,predict_decision_tree,predict_random_forest, 
                               predict_linear_regression,predict_logistic_regression_L2,
                               predict_logistic_regression_L1,predict_mlp]
    
    #Converting the above list of predictions into a dataframe, which will be used to train the stacking model.
    global stack_X
    stack_X=stack_X.append(build_data_frame(predict_list))
    
    #Building a list that contains all the raw features, used as cross validation data for the base models.
    global raw_features_X
    raw_features_X=raw_features_X.append(cross_val_X,ignore_index=True)
    
    #Storing the cross validation dataset labels in the variable stack_Y, which will be used later to train the stacking and blending models.
    global stack_Y
    stack_Y = cross_val_Y  

In [29]:
def print_metric_cross_val():
    
    #Calculating the average AUC across all the AUC computed on the cross validation folds.
    avg_linear_regression=np.mean(metric_linear_regression)
    avg_logistic_regression_L2=np.mean(metric_logistic_regression_L2)
    avg_logistic_regression_L1=np.mean(metric_logistic_regression_L1)
    avg_decision_tree=np.mean(metric_decision_tree)
    avg_random_forest=np.mean(metric_random_forest)
    avg_multi_layer_perceptron=np.mean(metric_multi_layer_perceptron)
    avg_gradient_boosting=np.mean(metric_gradient_boosting)
    
    #Printing the AUC for the base models.
    print (' AUC (Linear Regression)\n',avg_linear_regression)
    print (' AUC (Logistic Regression - L2)\n',avg_logistic_regression_L2)
    print (' AUC (Logistic Regression - L1)\n',avg_logistic_regression_L1)
    print (' AUC (Decision Tree)\n',avg_decision_tree)
    print (' AUC (Random Forest)\n',avg_random_forest)
    print (' AUC (Multi Layer Perceptron)\n',avg_multi_layer_perceptron)
    print (' AUC (Gradient Boosting - XGBoost)\n',avg_gradient_boosting)
    

In [30]:
def train_stack_blend():
    
    #Converting the above list of predictions and raw features (Concatenate) into a dataframe, which will be used to train the blending model.
    blend_X=pd.concat([raw_features_X, stack_X], axis=1,ignore_index=True)
    
    #Training the Stacking and Blending models parallely using the predictions of base models on the cross validation data.
    global stack
    global blend
    [stack,blend] = Parallel(n_jobs=-1)(delayed(model_function)(train_X,train_Y)for model_function,train_X,train_Y in [(train_stack_model,stack_X,stack_Y),(train_blend_model,blend_X,stack_Y)])
    

In [31]:
def re_initialize():
    #Initialzing the variables that will be used to calculate the area under the curve. (Test Data)
    metric_linear_regression=list()
    metric_logistic_regression_L2=list()
    metric_logistic_regression_L1=list()
    metric_decision_tree=list()
    metric_random_forest=list()
    metric_multi_layer_perceptron=list()
    metric_gradient_boosting=list()
    metric_stack=list()
    metric_blend=list()
    

In [32]:
def print_metric_test():
    print (' AUC (Linear Regression)\n',test_metric_linear_regression)
    print (' AUC (Logistic Regression - L2)\n',test_metric_logistic_regression_L2)
    print (' AUC (Logistic Regression - L1)\n',test_metric_logistic_regression_L1)
    print (' AUC (Decision Tree)\n',test_metric_decision_tree)
    print (' AUC (Random Forest)\n',test_metric_random_forest)
    print (' AUC (Multi Layer Perceptron)\n',test_metric_multi_layer_perceptron)
    print (' AUC (Gradient Boosting - XGBoost)\n',test_metric_gradient_boosting)
    print (' AUC (Stacking)\n',test_metric_stacking)
    print (' AUC (Blending)\n',test_metric_blending)

In [33]:
def test_data():
    
    global test_metric_linear_regression
    global test_metric_logistic_regression_L2
    global test_metric_logistic_regression_L1
    global test_metric_decision_tree
    global test_metric_random_forest
    global test_metric_multi_layer_perceptron
    global test_metric_gradient_boosting
    global test_metric_stacking
    global test_metric_blending

    
    #Training the base models, and calculating AUC on the cross validation data.
    #Selecting the data (Traing Data & Cross Validation Data)
    test_Y=test['y']
    test_X=test.drop(['y'],axis=1)
    
    #Computing the AUC and Predictions of all the base models on the cross validation data parallely.
    auc_predict_test=(Parallel(n_jobs=-1)(delayed(function)(test_X,test_Y) for function in cross_val_base_model_list))
    
    #Gradient Boosting (XGBoost)
    #The AUC error (Cross Validation Data)
    auc,predict_gradient_boosting=auc_predict_test[0][0],auc_predict_test[0][1]
    test_metric_gradient_boosting=(auc)
    
    #Multi Layer Perceptron
    #The AUC (Cross Validation Data)
    predict_mlp=list()
    auc,predict_multi_layer_perceptron=auc_predict_test[1][0],auc_predict_test[1][1]
    test_metric_multi_layer_perceptron=(auc)
    
    #predict_multi_layer_perceptron returns a list of lists containing the predictions, this cannot be converted to a dataframe.
    #This inner lists are converted to floats and then used to convert it to a dataframe.
    for i in predict_multi_layer_perceptron:
        predict_mlp.append(float(i))
    
    #Decision Tree)
    #The AUC (Cross Validation Data)
    auc,predict_decision_tree=auc_predict_test[2][0],auc_predict_test[2][1]
    test_metric_decision_tree=(auc)
    
    #Random Forest (Deafult=10 Trees)
    #The AUC (Cross Validation Data)
    auc,predict_random_forest=auc_predict_test[3][0],auc_predict_test[3][1]
    test_metric_random_forest=(auc)
    
    #Linear Regression
    #The AUC (Cross Validation Data)
    auc,predict_linear_regression=auc_predict_test[4][0],auc_predict_test[4][1]
    test_metric_linear_regression=(auc)
    
    #Logistic Regression (Default=l2)
    #The AUC (Cross Validation Data)
    auc,predict_logistic_regression_L1=auc_predict_test[5][0],auc_predict_test[5][1]
    test_metric_logistic_regression_L1=(auc)
    
    #Logistic Regression-L2
    #The AUC (Cross Validation Data)
    auc,predict_logistic_regression_L2=auc_predict_test[6][0],auc_predict_test[6][1]
    test_metric_logistic_regression_L2=(auc)
    
    #Building a list that contains all the predictions of the base models.
    predict_list=[predict_gradient_boosting,predict_decision_tree,predict_random_forest, 
                               predict_linear_regression,predict_logistic_regression_L2,
                               predict_logistic_regression_L1,predict_mlp]
    global test_stack_X
    global test_raw_features_X
    global test_blend_X
    
    test_stack_X=build_data_frame(predict_list)#Converting the list of predictions into a dataframe.
    test_raw_features_X=test_raw_features_X.append(test_X,ignore_index=True)
    test_blend_X=pd.concat([test_raw_features_X, test_stack_X], axis=1,ignore_index=True)#Converting the above list of predictions and raw features (Concatenate) into a dataframe

    #Computing the AUC and Predictions of the Stacking and Blending models on the test data parallely.
    auc_predict_test_second_level=Parallel(n_jobs=-1)(delayed(function)(test_X, test_Y) for function,test_X in ((cross_val_second_level_model[0],test_stack_X),(cross_val_second_level_model[1],test_blend_X)))

    #Stacking (XGBoost - Gradient Boosting)
    auc,predict_stack=auc_predict_test_second_level[0][0],auc_predict_test_second_level[0][1]
    test_metric_stacking=(auc)    

    #Blending (XGBoost - Gradient Boosting)
    auc,predict_blend=auc_predict_test_second_level[1][0],auc_predict_test_second_level[1][1]
    test_metric_blending=(auc)

In [34]:
def sample_generation(n):
    
    data_split()
    train_cross_val_base_models()
    print_metric_cross_val()
    train_stack_blend()
    test_data()
    print_metric_test()
    

In [ ]:
sample_generation(1)

In [35]:
(Parallel(n_jobs=-1)(delayed(sample_generation)(n) for n in range(3)))

/home/prajwal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
/home/prajwal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
/home/prajwal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1


Epoch 1/5
11648/18534 [=================>............] - ETA: 0s - loss: 0.3154 - acc: 0.8859
Epoch 2/5
 7040/18534 [==========>...................] - ETA: 0s - loss: 0.2799 - acc: 0.8898
Epoch 2/5
12800/18534 [===================>..........] - ETA: 0s - loss: 0.2642 - acc: 0.8924
Epoch 5/5
18534/18534 [==============================] - 1s - loss: 0.2622 - acc: 0.8945     


/home/prajwal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:25: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
/home/prajwal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:25: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
/home/prajwal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:25: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1


 AUC (Linear Regression)
 0.92818631168
 AUC (Logistic Regression - L2)
 0.929745016327
 AUC (Logistic Regression - L1)
 0.929774048395
 AUC (Decision Tree)
 0.91739325459
 AUC (Random Forest)
 0.906515838522
 AUC (Multi Layer Perceptron)
 0.876906113961
 AUC (Gradient Boosting - XGBoost)
 0.944818541713


/home/prajwal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1


...

/home/prajwal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:76: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1


 AUC (Gradient Boosting - XGBoost)
 AUC (Stacking)
 0.946455021463
 0.946918191896
 AUC (Blending)
 0.945845322893
 AUC (Linear Regression)
 0.931555144111
 AUC (Logistic Regression - L2)
 0.933099438653
 AUC (Logistic Regression - L1)
 0.933097669701
 AUC (Decision Tree)
 0.915389287231
 AUC (Random Forest)
 0.910664123308
 AUC (Multi Layer Perceptron)
 0.88511663286
 AUC (Gradient Boosting - XGBoost)
 0.946455021463
 AUC (Stacking)
 0.946918191896
 AUC (Blending)
 0.945845322893


/home/prajwal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:76: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1


 AUC (Linear Regression)
 0.931555144111
 AUC (Logistic Regression - L2)
 0.933099438653
 AUC (Logistic Regression - L1)
 0.933097669701
 AUC (Decision Tree)
 0.915389287231
 AUC (Random Forest)
 0.910664123308
 AUC (Multi Layer Perceptron)
 0.88511663286
 AUC (Gradient Boosting - XGBoost)
 0.946455021463
 AUC (Stacking)
 0.946918191896
 AUC (Blending)
 0.945845322893


[None, None, None]